<a href="https://colab.research.google.com/github/rg-smith/remote-sensing-hydro/blob/main/labs/lab3/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3

Use this notebook if you aren't able to get python working on your local computer. 

In this lab, you will use python (and a module called Google Earth Engine) to view satellite imagery, and access remotely sensed evapotranspiration datasets and compare them with precipitation and eddy flux evapotranspiration datasets.

Before starting the sections, we will import the modules we need, initialize Google Earth Engine, and define some functions.

The first block of code only needs to be run once, the first time you open your session (if you close out of the session and open again, you will need to run this again). Follow the prompts to initialize earth engine. You will be taken to a link where you need to give permission to link your google account with Google Earth Engine, then copy and paste some text below the code block.

In [ ]:
import ee
import folium
import numpy as np
import branca.colormap as cm
import pandas as pd
import matplotlib.pyplot as plt

!pip install rasterio geopandas
import rasterio
import geopandas as gpd

from google.colab import drive
drive.mount('/content/drive/')

if not ee.data._credentials:
    ee.Authenticate()
    ee.Initialize()

This block of code also only needs to be run once. It is defining a bunch of functions that you will use in this exercise. It is not important for you to understand the code here for this exercise.

In [ ]:
# functions needed for this lab (and some other useful ones that you can use if you're interested)

# to add a layer to our map:
def add_ee_layer(self, ee_object, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):  
            range = ee.Image(ee_object).reduceRegion(ee.Reducer.percentile([1, 99]),scale=10000)
            vals = range.getInfo()
            min=list(vals.items())[0][1]
            max=list(vals.items())[1][1]
            vis = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}

            map_id_dict = ee.Image(ee_object).getMapId(vis)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
            colormap = cm.LinearColormap(vmin=min,vmax=max,colors=['blue', 'white','red']).to_step(n=10)
            colormap.caption=name
            self.add_child(colormap)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except Exception as e:
        print("Could not display {}".format(name))
        print(e)

# convert ee image collection to pandas time series data frame
def ee_imgcoll_to_df(imagecollection, lat,lon):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    poi = ee.Geometry.Point(lon, lat)
    arr = imagecollection.getRegion(poi,500).getInfo()

    list_of_bands = imagecollection.first().bandNames().getInfo()

    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

# to convert a google earth engine image to a python array
def to_array(img,aoi):
  band_arrs = img.sampleRectangle(region=aoi,properties=['scale=1000'],defaultValue=-999)

  band_names=img.bandNames().getInfo()

  for kk in range(len(band_names)):
    if kk==0:
      dat1=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full=np.zeros((dat1.shape[0],dat1.shape[1],len(band_names)))
      dat_full[:,:,kk]=dat1
    else:
      dat=np.array(band_arrs.get(band_names[kk]).getInfo())
      dat_full[:,:,kk]=dat
  return(dat_full)

# to calculate an index
def getIndex(image,b1,b2):
  return image.normalizedDifference([b1, b2])

# to calculate a ratio
def getRatio(image1,image2):
  ratio=image1.divide(image2)
  return ratio

# to create a color map from a specific image
def getVisparams(image,aoi):
  range = image.reduceRegion(ee.Reducer.percentile([1, 99]),aoi,300)
  vals = range.getInfo()
  min=list(vals.items())[0][1]
  max=list(vals.items())[1][1]
  visParams = {'min': min, 'max': max, 'palette': ['0000FF', 'FFFFFF','FF0000']}
  return(visParams)

# to get the link to download an earth engine image
def getLink(image,aoi):
  link = image.getDownloadURL({
    'scale': 1000,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': aoi})
  print(link)

# create an earth engine geometry polygon
def addGeometry(min_lon,max_lon,min_lat,max_lat):
  geom = ee.Geometry.Polygon(
      [[[min_lon, max_lat],
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat]]])
  return(geom)

# load prism data
def get_prism_image(date1,date2,geometry):
  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m')
  prism_img = prism.filterDate(date1,date2).select('ppt').mean().clip(geometry)
  return(prism_img) # returns prism average monthly precipitation, in mm

# load landsat 8 data
def get_l8_image(date1,date2,geometry):
  l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')
  l8_img = l8.filterDate(date1,date2).mean().clip(geometry)
  return(l8_img)

def export_to_drive(raster,filename,foldername,geometry):
  # Export the image, specifying scale and region.
  task = ee.batch.Export.image.toDrive(**{
      'image': raster,
      'description': filename,
      'folder': foldername,
      'fileNamePrefix': filename,
      'scale': 1000,
      'region': geometry,
      'fileFormat': 'GeoTIFF',
      'formatOptions': {
        'cloudOptimized': 'true'
      },
  })
  task.start()

def get_elev(geometry):
  elev = ee.Image('USGS/NED').clip(geometry)
  return(elev)

def get_srtm(geometry):
  elev = ee.Image('USGS/SRTMGL1_003').clip(geometry)
  return(elev)

def get_gpm_image(date1,date2,geometry):
  gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V06')
  gpm_img = gpm.filterDate(date1,date2).select('precipitation').sum().multiply(24*365/12).clip(geometry)
  return(gpm_img) # returns total gpm precipitation in mm

def get_mod16ET(date1,date2,geometry):
  mod16 = ee.ImageCollection('MODIS/006/MOD16A2')
  mod16_img = mod16.filterDate(date1,date2).select('ET').sum().divide(10).clip(geometry)
  return(mod16_img)

def get_mod16PET(date1,date2,geometry):
  mod16 = ee.ImageCollection('MODIS/006/MOD16A2')
  mod16_img = mod16.filterDate(date1,date2).select('PET').sum().divide(10).clip(geometry)
  return(mod16_img)

# load sentinel 2 data
def get_s2_image(date1,date2,geometry):
    s2 = ee.ImageCollection('COPERNICUS/S2')
    s2_img = s2.filterDate(date1,date2).filterBounds(geometry).first().clip(geometry)
    return(s2_img)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Part 1: Viewing data for a given geometry and date

 

First, we will define the study area and time period and load some PRISM, GPM and elevation data.

In [ ]:
# create a bounding box that defines the study area
geom = addGeometry(-95, -85,30,40) # min long, max long, min lat, max lat

# define dates of interest (inclusive).
start = '2020-04-01'
end = '2021-04-01' #can go up to april 2021

# now get gpm precipitation over the same region for a specified time period
gpm_img = get_gpm_image(start,end,geom)

# now get MOD16 ET data over the same time period/region
et_img = get_mod16ET(start,end,geom)

pet_img = get_mod16PET(start,end,geom)

# get PET - ET
residual = pet_img.subtract(et_img)

Now, we will view the map. Scroll to the top so you can check/uncheck layers. Take a screenshot for your lab report.

In [ ]:
my_map = folium.Map(location=[35, -90], zoom_start=6)

# Add the land cover to the map object.
my_map.add_ee_layer(residual,'MOD16 PET - ET')
my_map.add_ee_layer(pet_img,'MOD16 PET')
my_map.add_ee_layer(et_img,'MOD16 ET')
my_map.add_ee_layer(gpm_img,'GPM precip')

my_map.add_ee_layer(geom,'bounding box')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

Like last lab, we will now export the data to your Google Drive. You can view it in rasterio on here, or download it to your computer and view in QGIS.

In [ ]:
export_to_drive(gpm_img,'lab3_gpm_precip','Colab Notebooks',geom) # raster, file name, folder name, geometry to clip it with
export_to_drive(et_img,'lab3_mod16_ET','Colab Notebooks',geom)
export_to_drive(pet_img,'lab3_moc16_PET','Colab Notebooks',geom)

**Part 2**

Now re-do this over your study area! 